## Cleaning safety data for Williamsburg Transportation study. 

#### This code is for cleaning and editing the collision data from the New York City Police Department. Many of the entries created by police officers were classified incorrectly or needed changes. 

In [ ]:
# Check to see they are all from the same year
# import libraries
import pandas as pd

crashes = 'collisions.csv'
# Read the filtered CSV file into a DataFrame
df_2023_check = pd.read_csv(crashes)

# Convert the 'CRASH DATE' column to datetime, allowing for mixed formats
df_2023_check['CRASH DATE'] = pd.to_datetime(df_2023_check['CRASH DATE'], errors='coerce')

# Check if there are any NaT values indicating parsing issues
if df_2023_check['CRASH DATE'].isna().any():
    print("There are some dates that could not be parsed.")
else:
    # Check if all dates are from 2022
    all_dates_2022 = df_2023_check['CRASH DATE'].dt.year.eq(2023).all()
    
    if all_dates_2022:
        print("All dates in the new CSV are from the year 2023.")
    else:
        print("Some dates in the new CSV are not from the year 2023.")

In [ ]:
# This code was for reclassifying a road file of New York City to make it joinable to the collisions database.
# I made the street names match the formatting so they could be cross referenced
import pandas as pd 
import os 
import geopandas as gpd

shapefilepath = 'roadshapefile.shp'
gdf = gpd.read_file(shapefilepath)

def transform_street_name(name):
    name = name.replace("NORTH", "N")
    name = name.replace("SOUTH", "S")
    name = name.replace("STREET", "ST")
    name = name.replace("AVENUE", "AVE")
    name = name.replace("PLACE", "PL")
    name = name.replace("ROAD", "RD")
    name = name.replace("DRIVE", "DR")
    return name

gdf['labelname'] = gdf['Street'].apply(transform_street_name)
    
# List of major roads
major_roads = ["METROPOLITAN AVENUE", "KINGSLAND AVENUE", "GREENPOINT AVENUE", "BERRY STREET", "GRAND STREET", "BUSHWICK AVENUE", "UNION AVENUE", "BROADWAY", "KINGSLAND AVENUE"]

# Create a new column to identify if a street is a major road
gdf['major_road'] = gdf['Street'].apply(lambda x: 1 if x in major_roads else 0)

gdf.to_file('newshapefile.shp')

In [3]:
import pandas as pd

# Load the CSV file
vehicle_crashes = 'collisions.csv'

vehicle_code1 = 'VEHICLE_TYPE_CODE_1'
vehicle_code2 = 'VEHICLE_TYPE_CODE_2'
vehicle_code3 = 'VEHICLE_TYPE_CODE_3'
vehicle_code4 = 'VEHICLE_TYPE_CODE_4'
vehicle_code5 = 'VEHICLE_TYPE_CODE_5'

df = pd.read_csv(vehicle_crashes)

value_counts_dict = {}

for vehicle_code in [vehicle_code1, vehicle_code2, vehicle_code3, vehicle_code4, vehicle_code5]:
    if vehicle_code in df.columns:
        
        distinct_values = df[vehicle_code].unique()
        value_counts = df[vehicle_code].value_counts()

        value_counts_dict[vehicle_code] = value_counts

        print(f"Distinct values for {vehicle_code}: {distinct_values}")
        print(f"Value counts for {vehicle_code}:\n{value_counts}\n")
    else:
        print(f"Column '{vehicle_code}' does not exist in the DataFrame")

vehicle_type_counts = pd.DataFrame(value_counts_dict).fillna(0).astype(int).reset_index()

vehicle_type_counts.columns = ['Vehicle Type', 'Count1', 'Count2', 'Count3', 'Count4', 'Count5']

vehicle_type_counts.to_csv('newcsvpath.csv', index=False)

Distinct values for VEHICLE_TYPE_CODE_1: ['Sedan' nan 'Tractor Truck Gasoline' 'E-Bike' 'Box Truck' 'Bike'
 'Motorcycle' 'Station Wagon/Sport Utility Vehicle' 'MILITARY' 'E-Scooter'
 'MOPED' 'Pick-up Truck' 'Taxi' 'Moped' 'Bus' 'PK' 'Flat Bed' 'Lift Boom'
 'Chassis Cab' 'Dump' 'Convertible' 'Carry All' '3-Door'
 'Tractor Truck Diesel' 'Tow' 'Garbage or Refuse' 'Minicycle'
 'Motorscooter' 'Van' 'LSV' 'Ambulance' 'Backhoe' 'Tanker' 'AMBULANCE'
 'Refrigerated Van' 'LIMO' 'van' 'Motorbike' 'Stake or Rack'
 'Concrete Mixer' 'Armored Truck' 'Trailer' 'FDNY FIRET' 'FDNY TRUCK'
 'Commercial' 'TRAILER' 'Beverage Truck' 'GARBAGE TR'
 'Tow Truck / Wrecker' 'Road Sweep' 'ambulance' 'Flat Rack']
Value counts for VEHICLE_TYPE_CODE_1:
VEHICLE_TYPE_CODE_1
Sedan                                  1298
Station Wagon/Sport Utility Vehicle    1037
Box Truck                               106
Bike                                     83
Pick-up Truck                            58
Motorcycle                    

#### This portion of the code takes a csv that contains each old name entered by the police officers and the new name that I want it to be reclassified as. It then creates new columns with the new classifications in the original csv with all the collision information.

In [ ]:

import pandas as pd 
import geopandas as gpd 

vehicle = r'C:\Users\Erin-PC\OneDrive - SOHO SOLUTIONS INC\Desktop\GIS Files_WilliamsburgTransportationStudy\Safety\Clipped to Study Area\collisions_clippedtostudy_2022.csv'
new_vehicle_codes = r'C:\Users\Erin-PC\OneDrive - SOHO SOLUTIONS INC\Desktop\GIS Files_WilliamsburgTransportationStudy\Safety\Clipped to Study Area\uniquevehiclenames_prefix.csv'
vehicle_df = pd.read_csv(vehicle)
new_vehicle_codes_df = pd.read_csv(new_vehicle_codes)

vehicle_code1 = 'VEHICLE_TYPE_CODE_1'
vehicle_code2 = 'VEHICLE_TYPE_CODE_2'
vehicle_code3 = 'VEHICLE_TYPE_CODE_3'
vehicle_code4 = 'VEHICLE_TYPE_CODE_4'
new_names_column = 'New_Name'
original_names_column = 'Original_Name'

vehicle_df = vehicle_df.merge(new_vehicle_codes_df[[original_names_column, new_names_column]],
                            how='left',
                            left_on=vehicle_code1,
                            right_on=original_names_column,
                            suffixes=('', '_1'))

vehicle_df = vehicle_df.merge(new_vehicle_codes_df[[original_names_column, new_names_column]],
                            how='left',
                            left_on=vehicle_code2,
                            right_on=original_names_column,
                            suffixes=('', '_2'))

vehicle_df = vehicle_df.merge(new_vehicle_codes_df[[original_names_column, new_names_column]],
                            how='left',
                            left_on=vehicle_code2,
                            right_on=original_names_column,
                            suffixes=('', '_3'))

vehicle_df = vehicle_df.merge(new_vehicle_codes_df[[original_names_column, new_names_column]],
                            how='left',
                            left_on=vehicle_code2,
                            right_on=original_names_column,
                            suffixes=('', '_4'))

# Drop the unnecessary columns from the dataframe
vehicle_df = vehicle_df.drop(columns=['Original_Name_1', 'Original_Name_2', 'Original_Name_3', 'Original_Name_4'])

# Save the updated data to a new CSV file
vehicle_df.to_csv(newcsv.csv)



In [1]:
# This piece of code is for creating a csv which each unique vehicle type that was entered by officers. This way, we can see where typos have come up.
import pandas as pd

# Path to the CSV file containing vehicle crash data
vehicle_crashes = 

# Define column names for vehicle types
vehicle_code1 = 'VEHICLE_TYPE_CODE_1'
vehicle_code2 = 'VEHICLE_TYPE_CODE_2'
vehicle_code3 = 'VEHICLE_TYPE_CODE_3'
vehicle_code4 = 'VEHICLE_TYPE_CODE_4'
vehicle_code5 = 'VEHICLE_TYPE_CODE_5'

# Read the CSV file into a DataFrame
df = pd.read_csv(vehicle_crashes)

# Get unique vehicle types and their counts from the first vehicle type column
value_counts1 = df[vehicle_code1].value_counts()

# Create a DataFrame for vehicle types and their counts
vehicle_type_counts = pd.DataFrame(value_counts1).reset_index()
vehicle_type_counts.columns = ['Vehicle Code', 'Count']

# Save the DataFrame to a CSV file
output_csv_path = 'output.csv'
vehicle_type_counts.to_csv(output_csv_path, index=False)

print("CSV file with vehicle codes and their counts has been created.")


CSV file with vehicle codes and their counts has been created.
